Hint Generation Tests
===

Goal: explain to the student why their answer is wrong.

Axis 1:
 - With revealing the right answer
 - Without revealing the right answer

Axis 2:
 - Using no lesson content? (Probably not, doesn't matter how good this is.)
 - Using lesson context only
 - Using larger Rori context
 - Using Rori + external math corpus context

Axis 3:
 - Single prompt, no CoT
 - Single CoT prompt + refinement prompt
 - Multi-stage refinment or retrieval

In [34]:
import json
import os
import time
from pathlib import Path
from pprint import pprint

import dotenv
import matplotlib.pyplot as plt
import numpy as np
import openai
import pandas as pd
import scipy
import tiktoken
from tqdm import tqdm

In [2]:
dotenv.load_dotenv("../.env")

True

In [3]:
os.environ["OPENAI_API_KEY"][:3]

'sk-'

In [4]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [5]:
data_dir = Path("../data")
assert data_dir.exists()
lesson_df = pd.read_pickle(data_dir / "derived" / "rori_lessons.pkl")
lesson_df.shape

(2570, 20)

In [6]:
lesson_df.sample(n=3)

,lesson_code,grade,domain,construct,subconstruct,skill,lesson_index,topic,short_description,lesson,question,answer,hint,has_image,section,question_index,is_explanation,needs_context,context_string,n_question_tokens
1415,G3.N1.2.2.5,3,N,N1,N1.2,N1.2.2,5,Find place values (5 of 5),Amazing! Let's convert 2 and 3 digit number fr...,None,500 + 50 + 3 = __53,5,The hundreds digit is missing.\nWhich digit sh...,False,1,5,False,False,,7.0
2088,G6.N1.2.2.6,6,N,N1,N1.2,N1.2.2,6,Find place values (6 of 7),In today's lesson we are going to learn about ...,None,8 hundred thousands = _____ tens,80000,The hundred thousands place value is four plac...,False,1,6,False,False,,6.0
2498,G7.N4.2.1.2,7,N,N4,N4.2,N4.2.1,2,Integer operations (2 of 5),Are you ready to have some fun today?\nWe goin...,None,64 - 99 = ___,-35,Subtract 64 from 99. \nThe answer has the sign...,False,1,6,False,False,,5.0


In [7]:
lesson_df[["question_index", "is_explanation", "section", "lesson", "question", "answer", "hint"]].sample(n=3)

,question_index,is_explanation,section,lesson,question,answer,hint
1341,8,False,1,None,"44, 34, 24, ___",14,Subtract 10 from 24.
711,0,True,1,Write *Yes* to proceed.,None,None,None
1845,11,False,2,None,Sixty thousand four hundred and one using digi...,T,The number has a 0 in thousands place and a 0 ...


In [8]:
lesson_codes_with_images = set()
for lesson_code, lesson in lesson_df.groupby("lesson_code"):
    has_image_explanation = ((lesson.is_explanation) & (lesson.has_image)).any()
    has_image_question = ((~lesson.is_explanation) & (lesson.has_image)).any()
    has_image = has_image_explanation or has_image_question
    lesson_codes_with_images.add(lesson_code)
len(lesson_codes_with_images)

225

In [10]:
sdf = lesson_df[(lesson_df.lesson_code.isin(lesson_codes_with_images)) & (lesson_df.grade.isin([6, 7, 8]))]
f"{len(sdf)} / {len(lesson_df)}", f"{sdf.lesson_code.nunique()} / {lesson_df.lesson_code.nunique()}"

('904 / 2570', '80 / 225')

In [22]:
lesson_codes = set(sdf.lesson_code)
n_lessons = 3
subset_lesson_codes = set(np.random.choice(np.array(list(lesson_codes)), size=n_lessons, replace=False))
sdf = sdf[sdf.lesson_code.isin(subset_lesson_codes)]
f"{len(sdf)} / {len(lesson_df)}", f"{sdf.lesson_code.nunique()} / {lesson_df.lesson_code.nunique()}"

('33 / 2570', '3 / 225')

In [53]:
message_lists = []

for lesson_code, lesson in sdf.groupby("lesson_code"):
    lesson_plan = lesson.iloc[0].short_description.strip() + "\n\n\n" + lesson.iloc[0].lesson.strip()
    index = 1
    row = lesson.iloc[index]
    while row.is_explanation:
        lesson_plan += "\n\n\n" + row.lesson.strip()
        index += 1
        row = lesson.iloc[index]
    messages = [
        {
            "role": "system",
            "content": "You are going to act as a maths teacher for a 13 year old student who is in grade 6.\n"
            "Your role is to introduce the student to a new topic using the following lesson plan delimited "
            'by triple quotes: """{lesson_plan}"""\n'
            "You will teach the lesson, presenting it in sections and waiting for the student "
            "to indicate they understand before proceeding. If the student has questions, provide "
            "short, encouraging, and educational clarifications.\nYour responses should be at the "
            "level of difficulty of a 12 year old.".format(lesson_plan=lesson_plan),
        }
    ]
    for lesson_component in lesson_plan.split("\n\n\n"):
        messages.extend(
            [
                {
                    "role": "assistant",
                    "content": lesson_component,
                },
                {
                    "role": "user",
                    "content": "Yes, I understand.",
                },
            ]
        )
    question_text = row.question + "\n"
    messages.extend(
        [
            {
                "role": "assistant",
                "content": question_text,
            },
            {
                "role": "user",
                "content": f"I don't understand why the answer is {row.answer}. Can you give me a hint?",
            },
        ]
    )
    message_lists.append(messages)

    pprint(messages)
    print()
len(message_lists)

[{'content': 'You are going to act as a maths teacher for a 13 year old '
             'student who is in grade 6.\n'
             'Your role is to introduce the student to a new topic using the '
             'following lesson plan delimited by triple quotes: """In today\'s '
             'lesson we are going to learn and practice long division of 3 '
             'digit and 4 digit numbers by 2 digit numbers\n'
             '\n'
             '\n'
             'The long division method helps to divide bigger numbers.\n'
             '\n'
             'I hope I was able to help you understand.\n'
             'Would you like to try some questions?\n'
             '\n'
             'Fill in the blanks.\n'
             '\n'
             'Type Yes to proceed."""\n'
             'You will teach the lesson, presenting it in sections and waiting '
             'for the student to indicate they understand before proceeding. '
             'If the student has questions, provide short, encourag

3

In [35]:
tokeniser = tiktoken.encoding_for_model("gpt-3.5-turbo-0613")
tokeniser

<Encoding 'cl100k_base'>

In [55]:
completions = []
for messages in message_lists:
    token_count_total = 0
    for message in messages:
        content = message["content"]
        token_count = len(tokeniser.encode(content))
        token_count_total += token_count
    print(token_count_total)
    if token_count_total > 2000:  # max of 4K (or 16K with the extended-context model)
        continue
    completion = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613", messages=messages)
    completions.append(completion)
len(completions)

266
299
278


3

In [61]:
for completion in completions:
    print(completion["choices"][0]["message"]["content"])
    print()

Of course! In the long division method, we start by dividing the first digit of the dividend (in this case, the first digit of 900) by the divisor. In this case, the first digit is 9. When 9 is divided by 10, what do you get?

Of course! To find the reciprocal of a fraction, you simply switch the numerator and denominator. 
So for the fraction 4/5, the reciprocal would be 5/4. 

Think of it as "flipping" the fraction over. The numerator becomes the denominator, and the denominator becomes the numerator. 

Give it a try and let me know if you have any more questions!

Of course! When dividing 4.5 by 9, think about how many times 9 can go into 4.5. Start by dividing the whole numbers and then consider the decimal part. In this case, 9 can go into 4.5 exactly 0 whole times. Now, when we bring down the decimal point in the quotient, what do you think the answer would be?

